## Main EDA file for teamwork - Team Phoenix
- Team members
    - Jack McCann (leader)
    - Nicole Muldowney
    - Teresa Whitesell
    - Ari Khursheed
    - Diego Alvarez
    - Lori Butler

## MVP

**Question from HCBB Presentation, Goal #2:**  
- Which procedures (HCPCS Codes) had the largest change in Average payment (using Average Medicare Allowed Amount, per pg 12 of HCBB presentation?
- Which procedures (HCPCS Codes) had the largest change in Utilization (using Number of Distinct Medicare Beneficiary/Per Day Services)?   
 
**Stretch goal:**   
Include Hospitals in analysis. Remember to convert old APCs in the 2015 hospital file to match the 2016-2017 APC codes.   
- APC is a group of related procedures (see pg 26 of HCBB presentation for more info).      
    - All procedures within an APC are paid at the same rate.  
    - Not every HCPCS/CPT code is assigned to an APC, but the most important ones are.  
    - APCs were renumbered in 2016.    

**MVP**  
- Static charts and/or interactive dashboard showing answers to questions:  
    - Largest Change in Ave. Payment over time  
    - Largest Change in Utilization  
        - Might also look at change in number of lives covered by Medicare as a whole, to see if there's useful information there (i.e., did utiliztaion go up or down at a faster rate than the number of covered beneficiaries?)   
- Creat dashboard that allows user to select   
    - A: specific HCPCS code/description (and maybe ACS for hospitals) change over the 3 years based on
    - B: amount, or utilization 

## Naming conventions

- **Columns:** Kept full name, replaced spaces and “/” with underscores. Otherwise kept the full column name without further edits

Dataframes:  
- **Initial DataFrames by year:**  
    - df_payments_20yy
- **Combined DataFrame** - concatenated, with separate row for each year (each NPI will have three rows):  
    - df_payments_combined
- **New Column for payment type description** (i..e, Doctor Only, Facility Only, or Doctor and Facility”
    - Payment_type   


- **New DataFrame to show average payment - concatenated** (year as value in ‘year’ column)
    - df_avg_pmt
- **New DataFrame to show average of the number_of_distinct_medicare_beneficiary_per_day_services**:
    - df_med_services_day   


- **New DataFrame to show average payment - pivoted** (year as column head)
    - df_pmt_pvt
- **Dataframe of unique beneficiaries per day pivoted by year**
    - df_bpd_pvt


## How we handled nulls

##### All years: Discovered 427 rows have null in last name. 
- Researched this info on data.gov visualization tool. (See notes under dropped columns)
- All other information was there (first name, middle initial). 
- Decided to KEEP these rows since all other columns that were meaningful to us were not null.

##### 2015 - Problem with single row
- Dropped one row that had irrelevant data in the last name field “CPT Copyright 2014….” and null values in all other columns.
- Not relevant to our project, so we dropped that column (done in prior notebeook, "step02..."

In [ ]:
import pandas as pd
import pickle
from glob import glob
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

## Read in Combined DataFrame

This combined df has already eliminted the one irrelevant row from 2015, and has added a column for payment_type (Doctor Only, Facility Only, Doctor & Facility).

In [ ]:
df_payments_combined = pd.read_pickle('../data/pickled_files/payments_combined.pkl')
print(df_payments_combined.shape)
df_payments_combined.head()

## EDA of combined file

In [ ]:
df_payments_combined.shape

In [ ]:
df_payments_combined.columns

In [ ]:
df_payments_combined.head()

In [ ]:
df_payments_combined.info()

In [ ]:
# To look ta count of null values

df_payments_combined.isnull().sum()


# Results are as expeceted. We researched the 427 last name/org name nulls as a team earlier
# They don't have last/org name, many had first name (which we didn't import), yet all of them
# had full data in all other fields. We chose to keep the 427 rows with null last names
# since none of our final reporting requires knowing the name.


In [ ]:
# To see the number of unique values of HCPCS codecodes  

count_of_codes = df_payments_combined.hcpcs_code.nunique()
print("There are", count_of_codes, "unique HCPCS codes in 2015-2017")

In [ ]:
# Statistics for number_of_services column. 
# Range from 2.4M to 7.1M min/max

# NOTABLE: 
#  Statistics for number_of_services column:  
#  Big difference between mean 2.4M and 50th percentile (median) 4.4M. 

df_payments_combined.number_of_services.describe()  

In [ ]:
df_payments_combined.number_of_medicare_beneficiaries.describe()

In [ ]:
df_payments_combined.number_of_distinct_medicare_beneficiary_per_day_services.describe()

In [ ]:
df_payments_combined.average_medicare_allowed_amount.describe()

In [ ]:
# Checking the min in above number...
# 6.03538e-05 = 6.035380 x 10^-05 = 0.00006035380
# Used converter here: https://www.calculatorsoup.com/calculators/math/scientific-notation-converter.php

df_payments_combined.average_medicare_allowed_amount.min()

In [ ]:
# Tried to make plot. Takes too long. This doesn't run. Takes too long
# df_payments_2015to2017.plot()

## Notable findings from EDA
### Update: These were all resolved/researched. None caused problems with the findings we are trying to discover.
- Statistics for number_of_services column:  
    - Big difference between mean (average) and median (50th percentile) in these columns:
        - number_of_services
        - number_of_medicare_beneficiaries
        - number_of_distinct_medicare_beneficiary_per_day_services
        - average_medicare_allowed_amount

    - Average medicare allowed amount's minimum is less that 1 cent. That seems odd.
    
===================================================================================


In [ ]:
df_payments_combined.number_of_services.describe()

In [ ]:
df_payments_combined.number_of_distinct_medicare_beneficiary_per_day_services.describe()

In [ ]:
df_payments_combined.average_medicare_allowed_amount.describe()

## Creating new df showing average payment by year, then by payment type, then by HCPCS code.  (Code from Nicole)

In [ ]:
df_avg_pmt = df_payments_combined.groupby(['year',
                                           'payment_type',
                                           'hcpcs_code']).average_medicare_allowed_amount.mean().to_frame()

In [ ]:
# To reset index

df_avg_pmt = df_avg_pmt.reset_index()

In [ ]:
df_avg_pmt.head()

In [ ]:
df_median_pmt = df_payments_combined.groupby(['year',
                                           'payment_type',
                                           'hcpcs_code']).average_medicare_allowed_amount.median().to_frame()

In [ ]:
# To reset index

df_median_pmt = df_median_pmt.reset_index()

### Creating pivot table from above, to get one row per HPCPS by year/pmt type

In [ ]:
pivot_index = ['payment_type',
               'hcpcs_code', 
               'average_medicare_allowed_amount'
              ]

pivot_cols = ['year']

In [ ]:
%%time
df_median_pmt_pvt = df_median_pmt.pivot_table(index = pivot_index, 
                                              columns = pivot_cols, 
                                              values = 'average_medicare_allowed_amount', 
                                              aggfunc=np.median)
df_median_pmt_pvt = df_median_pmt.reset_index()

### Creating new df showing average count (# distinct med. beneficiary/per day services) by year, then by payment type, then by HCPCS code.  (Code from Nicole)

In [ ]:
df_med_services_day = df_payments_combined.groupby(['year',
                                                    'hcpcs_code']).number_of_distinct_medicare_beneficiary_per_day_services.mean().to_frame().reset_index()

In [ ]:
df_med_services_day.head()

In [ ]:
# Still seeing wide variance between mean and median (50th percentile). 
# Need to look for outliers before plotting.

df_avg_pmt.describe()

In [ ]:
# Still seeing wide variance between mean and median (50th percentile). 
# Need to look for outliers before plotting.

df_med_services_day.describe()

### Started new notebook named step04_eda_and_analysis. 
- Worked with Teresa on that notebook, attempting to get to the point of calculating the variance between 2015 and 2017.